In [2]:
import torch
import torch.nn as nn
import string
import numpy as np
import random
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
import torch.optim as optim
from IPython.core.debugger import set_trace
import IPython
import matplotlib.pyplot as plt

In [3]:
def get_digits(timesteps):
    rand_indices = [random.randint(0, len(string.digits)-1) for _ in range(timesteps)]
    rand_nums = list(map(lambda x: int(string.digits[x]), rand_indices))
    sorted_rand_nums = sorted(rand_nums)
    
    return rand_nums, sorted_rand_nums

get_digits(8)

([7, 9, 1, 9, 3, 7, 8, 0], [0, 1, 3, 7, 7, 8, 9, 9])

In [4]:
def generate_training_data(timesteps, num_examples):
    x = list(); y = list()
    
    for _ in range(num_examples):
        digit, sorted_digit = get_digits(timesteps)
        x.append(digit); y.append(sorted_digit)
        
    random_id = random.randint(0, len(x))
    print(x[random_id], y[random_id])
    
    return x, y

In [5]:
def encode_data(x, y):
    features_dim = 10
    one_hot_x = F.one_hot(torch.tensor(x), num_classes=features_dim)
    one_hot_y = F.one_hot(torch.tensor(y), num_classes=features_dim)
    
    return one_hot_x.to(torch.float32), one_hot_y.to(torch.float32)

In [6]:
x, y = generate_training_data(8, 10000)
one_hot_x, one_hot_y = encode_data(x, y)
print(one_hot_x.dtype, one_hot_y.dtype)

val_x, val_y = generate_training_data(8, 1000)
val_x, val_y = encode_data(val_x, val_y)
print(val_x.shape, val_y.shape)

[1, 2, 3, 6, 8, 6, 3, 2] [1, 2, 2, 3, 3, 6, 6, 8]
torch.float32 torch.float32
[1, 5, 0, 5, 6, 6, 5, 7] [0, 1, 5, 5, 5, 6, 6, 7]
torch.Size([1000, 8, 10]) torch.Size([1000, 8, 10])


In [7]:
class CustomData(Dataset):
    def __init__(self, x_data, y_data):
        super().__init__()
        self.x_data = x_data
        self.y_data = y_data
        
    def __len__(self):
        return len(self.x_data)
    
    def __getitem__(self, idx):
        return self.x_data[idx], self.y_data[idx]
    
dataset = CustomData(one_hot_x, one_hot_y)
val_dataset = CustomData(val_x, val_y)

train_loader = DataLoader(dataset, batch_size=128, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=128, shuffle=True)

In [8]:
class AttentionModel(nn.Module):
    def __init__(self, encoder_dim, decoder_dim, input_features_dim, output_features_dim, timesteps, num_training):
        super().__init__()
        self.timesteps = timesteps
        self.input_features_dim = input_features_dim
        
        self.densor1 = nn.Linear(decoder_dim+(2*encoder_dim), 10)
        self.tanh = nn.Tanh()
        self.densor2 = nn.Linear(10, 1)
        self.softmax = nn.Softmax(dim=1)
        
        self.pre_attention_lstm = nn.LSTM(input_size=input_features_dim, hidden_size=encoder_dim, batch_first=True, bidirectional=True)
        self.post_attention_lstm = nn.LSTM(input_size=encoder_dim*2, hidden_size=decoder_dim, batch_first=True)
        self.output_layer = nn.Linear(decoder_dim, output_features_dim)
        
        
    def forward(self, x, s0, c0):
        # x-> (B, timesteps, features) -> (128, 10, 26)
        # s0, c0 -> (1, B, decoder_dim) -> (1, 128, 16)
        #set_trace()
        self.s = s0; self.c = c0
        encoder_hidden_states, (hn, cn) = self.pre_attention_lstm(x) #hidden_states: (B, timesteps, encoder_dim*2) -> (128, 10, 16)
        outputs = list()
        
        for t in range(self.timesteps):
            context = self.one_step_attention(encoder_hidden_states) # (B, timesteps, 2*encoder_dim)
            _, (self.s, self.c )= self.post_attention_lstm(context, (self.s, self.c)) # _ -> (B, timesteps, decoder_dim)
            output = self.output_layer(self.s) # (1, B, output_features_dim) -> (1, 128, 26)
            outputs.append(output) # (timesteps, B, output_features_dim) -> (10, 128, 26)
        
        return outputs
    
    def one_step_attention(self, encoder_hidden_states):
        # encoder_Hidden_states -> (B, timesteps, 2*encoder_dim) -> (128, 10, 16)
        # self.s -> (1, B, decoder_dim)
        # 1. first the hidden state for decoder must be repeated to match the hidden states of encoder
        # self.s -> (timesteps, B, decoder_dim) -> permute -> (B, timesteps, decoder_dim)
        # 2. then concatenate the hidden state for decoder and the hidden state for encoder -> (B, timesteps, 2*encoder_dim + decoder_dim)
        # pass it through the first dense layer
        # pass it through the second dense layer
        # use softmax to decide which hidden state of encoder is the most important
        # use dot product to find the important hidden state of encoder and feed it as input to the decoder
        hidden_decoder = self.s.repeat(self.timesteps, 1, 1).permute(1, 0, 2) # (B, timesteps, decoder_dim) -> (128, 10, 16)
        concat = torch.concatenate([encoder_hidden_states, hidden_decoder], dim=-1) # (B, timesteps, 2*encoder_dim + decoder_dim) -> (128, 10, 32)
        e = self.tanh(self.densor1(concat)) # (B, timesteps, 10) -> (128, 10, 10)
        energies = self.softmax(self.densor2(e)) # (B, timesteps, 1) -> (128, 10, 1)
        # let's if without permute in next step if the code converges
        energies = energies.repeat(1, 1, 10).permute(0, 2, 1) # (B, timesteps, 10) -> (B, 10, 10)
        context = torch.bmm(energies, encoder_hidden_states) # (B, timesteps, timesteps) @ (B, timesteps, 2*encoder_dim) -> (B, timesteps, 2*encoder_dim)
        
        return context
        
    def predict(self, x):
        assert len(x) == self.timesteps
        one_hot = F.one_hot(torch.tensor(x), num_classes=self.input_features_dim)
        s0 = torch.zeros(1, 1, decoder_dim); c0 = torch.zeros(1, 1, decoder_dim)
        one_hot = one_hot.unsqueeze(0)
        pred = self.forward(one_hot.to(torch.float32), s0, c0)
        pred = [torch.argmax(t, dim=-1).tolist()[0][0] for t in pred]
        return pred

In [9]:
def calculate_loss(y_true, y_pred):
    ce = nn.CrossEntropyLoss()
    total = 0
    for target, logit in zip(list(y_true), list(y_pred)):
        loss = ce(logit[0], target)
        total += loss
        
    return total

In [36]:
imp = list()

def hook_function(module, input, output):
    if not am.training:
        imp.append(output[0].flatten().tolist())

In [37]:
features = 10
encoder_dim = 8
decoder_dim = 16
input_features_dim = features
output_features_dim = features
timesteps = 8
num_training = 10000
epochs = 40

attention_dict = dict()

s0, c0 = torch.zeros(1, num_training, decoder_dim), torch.zeros(1, num_training, decoder_dim)

am = AttentionModel(encoder_dim, decoder_dim, input_features_dim, output_features_dim, timesteps, num_training)
opt = optim.Adam(am.parameters())

In [38]:
for name, layer in am.named_children():
    if name == 'softmax':
        print(layer)
        handle = layer.register_forward_hook(hook_function)

Softmax(dim=1)


In [39]:
for epoch in range(epochs):
    am.train()
    train_loss = list()
    for i, (inputs, target) in enumerate(train_loader):
        #set_trace()
        target = target.permute(1, 0, 2)
        batch_size = inputs.size(0)
        s0 = torch.zeros(1, batch_size, decoder_dim); c0 = torch.zeros(1, batch_size, decoder_dim)
        outputs = am(inputs, s0, c0)
        
        opt.zero_grad()
        
        total_loss = calculate_loss(target, outputs)
        train_loss.append(total_loss)
        
        total_loss.backward()
        opt.step()
        
    print(f'Epoch {epoch}:: Train Loss {torch.mean(torch.tensor(train_loss))}')
    
    am.eval()
    with torch.no_grad():
        
#         s0 = torch.zeros(1, 1000, decoder_dim); c0 = torch.zeros(1, 1000, decoder_dim)
#         out = am(val_x, s0, c0)
        
#         val_loss = calculate_loss(val_y.permute(1, 0, 2), out)
        
#         print(f'Epoch {epoch}:: Val Loss {val_loss}')
        test = [1, 9, 4, 3, 8, 7, 5, 6]
        #set_trace()
        pred = am.predict(test)
        attention_dict[f'Epoch {epoch}'] = np.array(imp)
        imp = list()
        print(f'Input {test} --> Output {pred}')
        print()
    

Epoch 0:: Train Loss 18.33650016784668
Input [1, 9, 4, 3, 8, 7, 5, 6] --> Output [7, 7, 7, 7, 7, 7, 7, 7]

Epoch 1:: Train Loss 15.879549026489258
Input [1, 9, 4, 3, 8, 7, 5, 6] --> Output [0, 3, 3, 5, 7, 8, 8, 8]

Epoch 2:: Train Loss 11.252341270446777
Input [1, 9, 4, 3, 8, 7, 5, 6] --> Output [1, 3, 4, 5, 6, 8, 8, 9]

Epoch 3:: Train Loss 8.750053405761719
Input [1, 9, 4, 3, 8, 7, 5, 6] --> Output [1, 3, 4, 5, 6, 7, 8, 9]

Epoch 4:: Train Loss 7.315134525299072
Input [1, 9, 4, 3, 8, 7, 5, 6] --> Output [1, 3, 4, 5, 6, 7, 8, 9]

Epoch 5:: Train Loss 6.429124355316162
Input [1, 9, 4, 3, 8, 7, 5, 6] --> Output [1, 3, 4, 5, 6, 7, 8, 9]

Epoch 6:: Train Loss 5.753997802734375
Input [1, 9, 4, 3, 8, 7, 5, 6] --> Output [1, 3, 4, 5, 6, 7, 8, 9]

Epoch 7:: Train Loss 5.24852991104126
Input [1, 9, 4, 3, 8, 7, 5, 6] --> Output [1, 3, 4, 5, 6, 7, 8, 9]

Epoch 8:: Train Loss 4.822076320648193
Input [1, 9, 4, 3, 8, 7, 5, 6] --> Output [1, 3, 4, 5, 6, 7, 8, 9]

Epoch 9:: Train Loss 4.5301446914672

In [12]:
test1 = [8, 8, 1, 7, 5, 4, 3, 3]; test2 = [8, 0, 1, 3, 2, 2, 9, 3]
heat_dict = dict()
for i, test in enumerate([test1, test2]):
    pred = am.predict(test)
    print(pred)
    print(len(imp))
    heat_dict[f'Test {i}'] = np.array(imp)
    imp = list()
    print(f'After adding to dictionary {len(imp)}')

[1, 3, 3, 4, 5, 7, 8, 8]
8
After adding to dictionary 0
[0, 1, 2, 2, 3, 3, 8, 9]
8
After adding to dictionary 0


In [48]:
images = list()
for key in attention_dict:
    plt.imshow(attention_dict[key])
    plt.xticks(range(len(test)), labels=test)
    plt.yticks(range(len(test)), labels=test)
    
    fig = plt.gcf()
    fig.canvas.draw()
    image = np.frombuffer(fig.canvas.tostring_rgb(), dtype='uint8')
    image = image.reshape(fig.canvas.get_width_height()[::-1] + (3,))
    plt.clf()
    
    images.append(image)

<Figure size 640x480 with 0 Axes>

In [52]:
import imageio

# images = list(attention_dict.values())
imageio.mimsave('/kaggle/working/plots.gif', images, duration=10)